#### Assignment 5 of parallel computing with dask

Author: Behzad Barati
- In this script I tried to implement Dask library functions for doing some text mining actions.
- Dataset is a collection of 2 million recipes and we only use 3 columns of it. dataset is availabe at https://recipenlg.cs.put.poznan.pl/ .
- The main script is available at https://github.com/BehzadBarati/Proramming1 . Since some functions (i.e. translate) is not availabe in dask I had to change workflow and make new functions to practice with Dask.

In [1]:
#Import libraries

import numpy as np
import pandas as pd
import re

import dask
from dask.distributed import Client
import dask.dataframe as dd


In [2]:
#make a client to manage our parallel computation

client = Client()
client

Client Scheduler: tcp://127.0.0.1:55914 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.06 GB


### Read csv with pandas and Dask

In [3]:
%%time

# Read 3 columns of file

pdf = pd.read_csv('./Recipe_data/RecipeNLG.csv', skipinitialspace=True, usecols=['title', 'NER'])
pdf.head()

Wall time: 11.5 s


,title,NER
0,No-Bake Nut Cookies,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [4]:
%%time

# Read 3 columns of file

ddf = dd.read_csv('./Recipe_data/RecipeNLG.csv', skipinitialspace=True, usecols=['title', 'NER'])
ddf.compute()

Wall time: 6.94 s


,title,NER
0,No-Bake Nut Cookies,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...
43680,Sunny's Fake Crepes,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
43681,Devil Eggs,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
43682,Extremely Easy and Quick - Namul Daikon Salad,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
43683,Pan-Roasted Pork Chops With Apple Fritters,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [5]:
#Our 36 partitions has Pandas dataframe type. lenght of each partiotion is as follows

ddf.map_partitions(len).compute()

0     95525
1     95572
2     95777
3     95918
4     95773
5     95888
6     96173
7     96148
8     96062
9     69258
10    53425
11    52605
12    52763
13    52675
14    52701
15    52477
16    50320
17    49098
18    40897
19    42653
20    48829
21    51955
22    46016
23    49727
24    51741
25    50608
26    50610
27    50766
28    50667
29    50822
30    50886
31    50890
32    50647
33    50767
34    50818
35    43685
dtype: int64

Lets see what are our data and what is the total number of recipes. it has more than 2 million records.

In [6]:
%%time

print('Number of recipes in dataset: ', len(pdf))
print('last 5 recipes:')
pdf.tail(5)

Number of recipes in dataset:  2231142
last 5 recipes:
Wall time: 0 ns


,title,NER
2231137,Sunny's Fake Crepes,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,Devil Eggs,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."
2231141,Polpette in Spicy Tomato Sauce,"[""ground veal"", ""sausage"", ""bread crumbs"", ""mi..."


In [7]:
%%time

print('Number of recipes in dataset: ', len(ddf))
print('last 5 recipes:')
ddf.tail(5)

Number of recipes in dataset:  2231142
last 5 recipes:
Wall time: 4.63 s


,title,NER
43680,Sunny's Fake Crepes,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
43681,Devil Eggs,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
43682,Extremely Easy and Quick - Namul Daikon Salad,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
43683,Pan-Roasted Pork Chops With Apple Fritters,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."
43684,Polpette in Spicy Tomato Sauce,"[""ground veal"", ""sausage"", ""bread crumbs"", ""mi..."


#### Apply functions to both pdf (Pandas dataframe) and ddf (Dask dataframe)

In [12]:
useless_chars =[] 

#def censor(val, char):
#    # remove meaningless characters from words in list
#    return re.sub(char, '', val)

def split(s):
    # make characters capital.
    return s.split(',') 

def counter(df):
    return len(df)

def strp(listt):
    return [val.strip() for val in listt]


In [13]:
%%time

#pdf['NER'] = map(censor, )
pdf['NER'] = pdf['NER'].map(split)
pdf['NER_cont'] = pdf['NER'].map(counter)
pdf['NER'] = pdf['NER'].map(strp)

Wall time: 15.3 s


In [14]:
%%time

ddf['NER'] = ddf['NER'].map(split)
ddf['NER_cont'] = ddf['NER'].map(counter)
ddf['NER'] = ddf['NER'].map(strp)

Wall time: 8.95 ms


#### Apply functions which are capable of parallel computing

In order to have a guess on where the recipe belongs to, I used a function from GeoText library. This functions accepts a string of all capitalized words seprated by comma. So, we need to make all words in 'title' and 'NER' column captial and concatenate them into a new column ('NERtitle').    

For practicing prarallel computing of dask, I need to read orginal file once again.

In [2]:
pdf = pd.read_csv('./Recipe_data/RecipeNLG.csv', skipinitialspace=True, usecols=['title', 'NER'])
ddf = dd.read_csv('./Recipe_data/RecipeNLG.csv', skipinitialspace=True, usecols=['title', 'NER'])

In [3]:
def cap(listt):
    # make characters capital.
    return [val.capitalize() for val in listt]

def split1(s):
    # make characters capital.
    return s.split(',') 

def split2(s):
    #make our title string split into words
    return s.split(' ')

def join1(listt):
    return ', '.join(listt)

def concat(s1, s2):
    return s1 + ', ' + s2

def censor(s):
    # remove meaningless character from words in list
    s = re.sub('"', '', s)
    s = re.sub('\[', '', s)
    s = re.sub(']', '', s)
    return re.sub("''", '', s)

def strp(listt):
    return [val.strip() for val in listt]

In [12]:
series1 = pdf['NER']
series2 = pdf['title']

In [17]:
%%time

series1 = series1.map(censor)
series1 = series1.map(split1)
series1 = series1.map(strp)
series1 = series1.map(cap)
series1 = series1.map(join1)

series2 = series2.map(split2)
series2 = series2.map(cap)
series2 = series2.map(join1)

pdf['NERtitle'] = series2 + series1
pdf.head()

Wall time: 45.1 s


,title,NER,NERtitle
0,No-Bake Nut Cookies,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...","No-bake, Nut, CookiesBrown sugar, Milk, Vanill..."
1,Jewell Ball'S Chicken,"[""beef"", ""chicken breasts"", ""cream of mushroom...","Jewell, Ball's, ChickenBeef, Chicken breasts, ..."
2,Creamy Corn,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","Creamy, CornFrozen corn, Cream cheese, Butter,..."
3,Chicken Funny,"[""chicken"", ""chicken gravy"", ""cream of mushroo...","Chicken, FunnyChicken, Chicken gravy, Cream of..."
4,Reeses Cups(Candy),"[""peanut butter"", ""graham cracker crumbs"", ""bu...","Reeses, Cups(candy), , Peanut butter, Graham c..."


#### Here, I tried to do the same as above but with dask library

In [18]:
@dask.delayed
def cap(listt):
    # make characters capital.
    return [val.capitalize() for val in listt]

@dask.delayed
def split1(s):
    # make characters capital.
    return s.split(',') 

@dask.delayed
def split2(s):
    #make our title string split into words
    return s.split(' ')

@dask.delayed
def join1(listt):
    return ', '.join(listt)

@dask.delayed
def concat(s1, s2):
    return s1 + ', ' + s2

@dask.delayed
def censor(s):
    # remove meaningless character from words in list
    s = re.sub('"', '', s)
    s = re.sub('\[', '', s)
    s = re.sub(']', '', s)
    s = re.sub("''", '', s)
    return s

@dask.delayed
def strp(listt):
    return [val.strip() for val in listt]


In [19]:
series11 = ddf['NER']
series22 = ddf['title']

In [20]:
%%time

series11 = series11.map(censor)
series11 = series11.map(split1)
series11 = series11.map(strp)
series11 = series11.map(cap)
series11 = series11.map(join1)

series22 = series22.map(split2)
series22 = series22.map(cap)
series22 = series22.map(join1)

ddf['NERtitle'] = series22 + series11

ValueError: Metadata inference failed in `map`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ValueError('Cannot infer dataframe metadata with a `dask.delayed` argument')

Traceback:
---------
  File "C:\Users\Behzad\anaconda3\lib\site-packages\dask\dataframe\utils.py", line 174, in raise_on_meta_error
    yield
  File "C:\Users\Behzad\anaconda3\lib\site-packages\dask\dataframe\core.py", line 5165, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\Users\Behzad\anaconda3\lib\site-packages\dask\dataframe\core.py", line 5145, in _extract_meta
    return tuple([_extract_meta(_x, nonempty) for _x in x])
  File "C:\Users\Behzad\anaconda3\lib\site-packages\dask\dataframe\core.py", line 5145, in <listcomp>